In [1]:
import base64
import re
import os

## Задание 1.

Напишите функцию конвертации текста, представленном в шестнацеричном виде, в base64.

*Пример:*

Входные данные:

49276d207374756479696e672043727970746f677261706879206c696b6520436c6175646520456c776f6f64205368616e6e6f6e21

Выходные данные: 

SSdtIHN0dWR5aW5nIENyeXB0b2dyYXBoeSBsaWtlIENsYXVkZSBFbHdvb2QgU2hhbm5vbiE=

In [2]:
in_str = "49276d207374756479696e672043727970746f677261706879206c696b6520436c6175646520456c776f6f64205368616e6e6f6e21"
out_str = b"SSdtIHN0dWR5aW5nIENyeXB0b2dyYXBoeSBsaWtlIENsYXVkZSBFbHdvb2QgU2hhbm5vbiE="

In [3]:
def hex_str_to_str(input_str):
    result = ""
    for i in range(0, len(input_str), 2):
        ch16 = input_str[i:i+2]
        ch10 = int(ch16, 16)
        ch = chr(ch10)
        result += ch
    return result

def hex_str_to_base64(input_str):
    return base64.b64encode(hex_str_to_str(input_str).encode('ascii'))

In [4]:
in_str_proc = hex_str_to_base64(in_str)

In [5]:
in_str_proc == out_str

True

In [6]:
base64.b64decode(out_str)

b"I'm studying Cryptography like Claude Elwood Shannon!"

In [7]:
base64.b64decode(in_str_proc)

b"I'm studying Cryptography like Claude Elwood Shannon!"

## Задание 2.

Напишите функцию, которая принимает в качестве параметров два буфера одинаковой длины и производит операцию XOR над ними.

*Пример:*

Входные данные:

506561636520416c6c204f7665722054686520576f726c64

XOR

4949544353551c0111001f010100061a021f010100061a02

=

192C352036755D6D7D2050776472264E6A7A21566F747666

In [8]:
b1 = "506561636520416c6c204f7665722054686520576f726c64"
b2 = "4949544353551c0111001f010100061a021f010100061a02"
out = "192C352036755D6D7D2050776472264E6A7A21566F747666"

In [9]:
hex_str_to_str(b1)

'Peace All Over The World'

In [10]:
def xor_two_hex_str(b1, b2):
    assert len(b1) == len(b2)
    result = ""
    for i in range(len(b1)):
        result += f'{int(b1[i], 16) ^ int(b2[i], 16):x}'
    return result.upper()

In [11]:
result = xor_two_hex_str("506561636520416c6c204f7665722054686520576f726c64", "4949544353551c0111001f010100061a021f010100061a02")

In [12]:
out == result

True

## Задание 3.

**(не использовать метод полного перебора ключа)**

Дана строка закодированная в 16-виде. Данную строку получили путем операции XOR некоторого текста с одним символом. Расшифруйте это сообщение. 

*Пример:*

Входные данные:

19367831362e3d2b2c353d362c783136783336372f343d3c3f3d7839342f39212b782839212b782c303d783a3d2b2c7831362c3d2a3d2b2c

Стадии выполнения задания:

1: выполните задание без программной автоматизации - "на листочке".  
2: напишите программу дешифратор.

In [13]:
in_str = "19367831362e3d2b2c353d362c783136783336372f343d3c3f3d7839342f39212b782839212b782c303d783a3d2b2c7831362c3d2a3d2b2c"

In [14]:
# Делаем предположение, что в строке самый часто-встречающийся символ - это 'e' (самый часто-встречающийся символ в англ. языке)
# https://en.wikipedia.org/wiki/Letter_frequency
symb = f"{ord('e'):x}"

In [15]:
# Находим самый часто-встречающийся символ в зашифрованной строке
in_str_symb = [in_str[i:i+2] for i in range(0, len(in_str), 2)]
unique_symb = list(set(in_str_symb))
count_values = [in_str_symb.count(i) for i in unique_symb]
symb_decode = unique_symb[count_values.index(max(count_values))]

In [16]:
# Находим ключевой символ, путём повторного применения xor
symb_key = f'{int(symb_decode, 16) ^ int(symb, 16):x}'

In [17]:
def xor_hex_str_and_symb(hex_str, hex_symb):
    result = ""
    for i in range(0, len(hex_str), 2):
        result += f'{int(hex_str[i:i+2], 16) ^ int(hex_symb, 16):x}'
    return result.upper()

In [18]:
result = xor_hex_str_and_symb(in_str, symb_key)

In [19]:
hex_str_to_str(result)

'An investment in knowledge always pays the best interest'

## Задание 4.
В качестве входных данных дается файл со строками в hex. Одна из этих строк зашифрована с помощью XOR на одно-символьном ключе. Найдите и расшифруйте эту строку.

In [20]:
data_dir = "data/task4"
data_files = os.listdir(data_dir)
data_files.sort()

In [21]:
def brut_search_encoded_str_and_decode(data, max_error=1):
    reg = re.compile(u"[^a-zA-Z ?!-.',]")
    for i, line in enumerate(data):        
        symb_count = len(line) / 2
        for key in range(256):
            result_str = hex_str_to_str(xor_hex_str_and_symb(line, f"{key:x}"))
            result_str_reg = reg.sub('', result_str)
            if abs(len(result_str_reg) - symb_count) <= max_error:
                return i, chr(key), result_str_reg
    return None

In [22]:
for data_file in data_files:
    with open(os.path.join(data_dir, data_file), 'r') as f:
        data = f.read()
    data = data.split('\n')
    data = [d for d in data if d]
    result = brut_search_encoded_str_and_decode(data)
    if not result:
        print(f"file: {data_file} string not found")
        continue
    print(f"file: {data_file}  string_line: {result[0]}  key_symbol: {result[1]}  decoded_string: {result[2]}")

file: task140.input  string_line: 373  key_symbol: a  decoded_string: Kakim umnym ja sebja chuvstvuu
file: task141.input  string_line: 170  key_symbol: 5  decoded_string: Now that the party is jumping
file: task142.input  string_line: 373  key_symbol: a  decoded_string: Kakim umnym ja sebja chuvstvuu


## Задание 5.
Реализуйте шифрование XOR'ом с повторяющимся ключом.


*Пример:*

Входные данные: *data/task5/task15.input*

Ключ: Shannon

Результат: 00000000000000730b0e001a1d07311d150b0a4f1a3c4815060b4f083a0d0d0a4e0008730b13171e1b0f3d090d171d061d730e0e1c4e010f27010e000f034e370d070b001c0b730c141c070109733f0e1c020b4e0409134e272642736208000d031b37010f094e07072048030f1d060d731f0e1c054f013d4802010a0a0c210d000507010973090f0a4e1c0b301d130b4e1b0b3f0d020103021b3d01020f1a06013d1b4f

In [153]:
out_str_hex = "00000000000000730b0e001a1d07311d150b0a4f1a3c4815060b4f083a0d0d0a4e0008730b13171e1b0f3d090d171d061d730e0e1c4e010f27010e000f034e370d070b001c0b730c141c070109733f0e1c020b4e0409134e272642736208000d031b37010f094e07072048030f1d060d731f0e1c054f013d4802010a0a0c210d000507010973090f0a4e1c0b301d130b4e1b0b3f0d020103021b3d01020f1a06013d1b4f"
key = "Shannon"

In [154]:
def str_to_hex(in_str):
    result = ""
    for i in in_str:
        ch = f"{ord(i):x}"
        if len(ch) == 1:
            ch = f'0{ch}'
        result += ch
    return result

In [155]:
with open('data/task5/task15.input', 'r') as f:
    in_str = f.read()

in_str_hex = str_to_hex(in_str)
key_hex = str_to_hex(key)
key_hex = (key_hex * round((len(in_str_hex) / len(key_hex)) + 0.5))[:len(in_str_hex)]

In [157]:
result = xor_two_hex_str(in_str_hex, key_hex)

In [158]:
result == out_str_hex.upper()

True

## Задание 6.
Задание: Расшифруйте данные зашифрованные XOR'ом с повторяющимся ключом.  
Входные данные: *data/task6/\**

In [5]:
def hamming_distance(in_byte_1, in_byte_2):
    dist = 0
    for b1, b2 in zip(in_byte_1, in_byte_2):
        diff = b1 ^ b2
        dist += sum([1 for bit in bin(diff) if bit == '1'])
    return dist


def xor_bytеs_and_byte(byte_str, byte_symb):
    result = b''
    for i in range(len(byte_str)):
        result += bytes([byte_str[i] ^ byte_symb])
    return result


def bytes_to_str(byte_str):
    result = ""
    for i in range(len(byte_str)):
        result += chr(byte_str[i])
    return result


def xor_two_bytes(b1, b2):
    assert len(b1) == len(b2)
    result = b''
    for i in range(len(b1)):
        result += bytes([b1[i] ^ b2[i]])
    return result


def keysize_xor_duplecate_key(data_byte, max_keysize=41):
    avg_dist = []

    for keysize in range(2,max_keysize):
        distances = []
        chunks = [data[i:i+keysize] for i in range(0, len(data), keysize)]
        while True:
            try:
                chunk_1 = chunks[0]
                chunk_2 = chunks[1]
                dist = hamming_distance(chunk_1, chunk_2)
                distances.append(dist/keysize)
                del chunks[0]
                del chunks[1]
            except Exception as e:
                break

        result = { 'key': keysize, 'score': sum(distances) / len(distances) }
        avg_dist.append(result)

    return sorted(avg_dist, key=lambda x: x['score'])


def brut_xor_duplecate_key(data_byte):
    key_sizes = keysize_xor_duplecate_key(data)
    text_reg = re.compile(u"[^a-zA-Z0-9?!\-_.,:;'`\n\t\v ]")
    freq_symb = [' ', 'e', 'a', 'o', 'i', 'u', 't', 'n', 's', 'h']
    key = b''
    for k in key_sizes:
        
        possible_key_len = k['key']
        for i in range(possible_key_len):
            block = b''
            for j in range(i, len(data), possible_key_len):
                block += bytes([data[j]])

            unique_symb = list(set(block))
            count_values = [block.count(i) for i in unique_symb]
            symb_decode = unique_symb[count_values.index(max(count_values))]

            k = b""
            for symb in freq_symb:
                symb_key = symb_decode ^ ord(symb)

                result_bytes = xor_bytеs_and_byte(block, symb_key)
                result_str = bytes_to_str(result_bytes)
                result_str_reg = text_reg.sub('', result_str)

                if len(result_str_reg) == len(result_str):
                    k = symb_key
                    break
                    
            if not k:
                key = b''
                break
            key += bytes([k])
            
        if key:
            return key
    
    return None

In [18]:
data_dir = "data/task6"
data_files = os.listdir(data_dir)
data_files.sort()

for data_file in data_files:
    
    with open(os.path.join(data_dir, data_file), 'r') as f:
        data = base64.b64decode(f.read())

    key = brut_xor_duplecate_key(data)
    key_repeat = (key * round((len(data) / len(key)) + 0.5))[:len(data)]
    result_bytes = xor_two_bytes(data, key_repeat)
    result_str = bytes_to_str(result_bytes)
    print(f"file: {data_file}  key: {key}\n")
    print(result_str)
    print("------------------------------------------\n")

file: task160.input  key: b'Robert Laurence Binyon'

How dark, how quiet sleeps the vale below!
In the dim farms, look, not a window shines:
Distantly heard among the lonely pines,
How soft the languid autumn breezes flow
Past me, and kiss my hair, and cheek, and mouth!
Half--veiled is the calm sky:
Jupiter's kingly eye
Alone glows full in the unclouded South.

Alas! and can sweet Night avail to heal
Not one of the world's wounds? Must I, even here,
Still listen with the mind's too wakeful ear
To that sad sound, which in my flesh I feel;
Sound of unresting, unrejoicing feet,
With feverish steps or slow
For ever, to and fro,
Pacing the gay, thronged, friendless, stony street?

Nature is free; but Man the eternal slave
Of care and passion. Must I deem that true?
With fields and quiet have we nought to do,
Because our spirits for ever crave and crave,
And never found their satisfaction yet?
World, is thy heart so cold,
So deeply weary and old,
That thy sole business is but to forget?

No,